In [8]:
import numpy
testL=['a','b','c','d','e']
print(testL)
print(testL[:3])
print(testL[3:])

['a', 'b', 'c', 'd', 'e']
['a', 'b', 'c']
['d', 'e']


In [ ]:
###爬取宗旨就是：：： 
#（1）先读取  县数据表county_list  （id  county_name  city_id）
    #正常情况下要通过    city_id获取city_name   然后 http://www.a-hospital.com/w/ + city_name+county_name + 医院列表
    #非正常情况：  （1）url不一致：   http://www.a-hospital.com/w/ + county_name + 医院列表
                   #(2)页面不存在    没数据，直接为空，不写入数据库
    

In [3]:
import requests
from bs4 import BeautifulSoup

import pymysql
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', 
                       passwd='', db='jiadesen', charset='utf8')
cursor = conn.cursor()

sql = "SELECT * FROM county_list"
cursor.execute(sql)
#county_all_info  内容包括(id,city_id,county_name)
county_all_info = cursor.fetchall()
#print(county_all_info)
print(len(county_all_info))


#county_all_info[:1]  测试第一个朝阳区   可以爬取该页面  ------------最正常的一般页面   必须写成【：1】这种形式
#county_all_info[2790:2971]    西藏自治区    日喀则地区    日喀则市--------------url只含有county_name   
                                #必须写成【2790:2791】   否则下面出错   each_info[0]会出错，类型估计不一致
    
                                #重庆市  永川市    county_all_info【43:44】     解决方案是添加了if  判断条件  判断value是否存在
#county_all_info[2266:2267]      江西省   景德镇市   珠山区-------------相应页面不存在的情况  ：：  直接不做处理，即没有任何东西
    

#for each_info in county_all_info[2790:2791]:
#for each_info in county_all_info[43:44]:
#for each_info in county_all_info[2266:2267]:
for each_info in county_all_info[2559:]:
    tt_id = each_info[0]
    tt_city_id = each_info[1]
    tt_county_name = str(each_info[2])
    sql2 = "SELECT city_name FROM city_list WHERE id = %d "  %tt_city_id
    cursor.execute(sql2)
    tt_city_name = str(cursor.fetchone()[0])
    print(tt_id,tt_city_id,tt_county_name,tt_city_name)
    
    
    str1 = 'http://www.a-hospital.com/w/'
    str2 = tt_city_name + tt_county_name
    str3 = '医院列表'
    url = str1 + str2 + str3
    print(url)
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    names = soup.select('ul > li > b > a')
    #print(names)
    informations = soup.select('ul > li > ul')
    #print(shengshi)
    print(len(names))
    print(len(informations))
    
    #################################################
    #需要添加第一个错误处理-----url直接是county_name
    if len(names) == 0:
        url = str1 + tt_county_name + str3
        print(url)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'lxml')
        names = soup.select('ul > li > b > a')
        #print(names)
        informations = soup.select('ul > li > ul')
        #print(informations)
        print(len(names))
        print(len(informations))
        
        ### len(names)==0  针对该县页面不存在情况，直接不做处理即可
        if len(names)!=0:
            dict = {'hospital_name': 'None' ,'hospital_rank':'None ','hospital_property':'None'};
            #list=[]
            #print(dict)
            #print(list)

            for i in range(len(names)):
                name = names[i].find_parent('b').get_text()
                #print(name)
                dict['hospital_name'] = name
                #print(dict['hospital_name'])
                info = informations[i].select('li')
                for li in info:
                    if li.b:
                        title = li.b.get_text()
                        if title == "医院等级":
                            value = li.contents[1].replace('：', '')
                            if not value:
                                value = li.a.contents[0]
                            #print(li.a.contents[0])
                            #value = li.a.contents[0]
                            #print(title,":",value)
                            dict['hospital_rank'] = value
                            #print(dict['hospital_rank'])
                        elif title == "经营方式":
                            value2 = li.contents[1].replace('：', '')
                            if not value2:
                                value2 = li.a.contents[0]
                            #print(title,":",value2)
                            dict['hospital_property'] = value2
                            #print(dict['hospital_property'])
                
                print(dict)
                #print(tt_id)
                
                insert_sql = "INSERT INTO hospital_information(county_id, hospital_name, hospital_rank, hospital_property) VALUES(%s,%s,%s,%s)"
                #print(insert_sql,(tt_id,dict['hospital_name'],dict['hospital_rank'],dict['hospital_property']))
                cursor.execute(insert_sql,(tt_id,dict['hospital_name'],dict['hospital_rank'],dict['hospital_property']))
                conn.commit()
                
                dict = {'hospital_name': 'None' ,'hospital_rank':'None ','hospital_property':'None'}
            
        
        
    
    ###################################################
    
    else:
    
        dict = {'hospital_name': 'None' ,'hospital_rank':'None ','hospital_property':'None'};
        #list=[]
        #print(dict)
        #print(list)

        for i in range(len(names)):
            name = names[i].find_parent('b').get_text()
            #print(name)
            dict['hospital_name'] = name
            #print(dict['hospital_name'])
            info = informations[i].select('li')
            for li in info:
                if li.b:
                    title = li.b.get_text()
                    if title == "医院等级":
                        value = li.contents[1].replace('：', '')
                        if not value:
                            value = li.a.contents[0]
                        #print(title,":",value)
                        dict['hospital_rank'] = value
                        #print(dict['hospital_rank'])
                    elif title == "经营方式":
                        value2 = li.contents[1].replace('：', '')
                        if not value2:
                            value2 = li.a.contents[0]
                        #print(title,":",value2)
                        dict['hospital_property'] = value2
                        #print(dict['hospital_property'])
            
            print(dict)
            #print(tt_id)
          
            insert_sql = "INSERT INTO hospital_information(county_id, hospital_name, hospital_rank, hospital_property) VALUES(%s,%s,%s,%s)"
            #print(insert_sql,(tt_id,dict['hospital_name'],dict['hospital_rank'],dict['hospital_property']))
            cursor.execute(insert_sql,(tt_id,dict['hospital_name'],dict['hospital_rank'],dict['hospital_property']))
            conn.commit()
            
            dict = {'hospital_name': 'None' ,'hospital_rank':'None ','hospital_property':'None'}
        


2863
2560 303 秦安县 天水市
http://www.a-hospital.com/w/天水市秦安县医院列表
3
3
{'hospital_rank': '二级甲等', 'hospital_name': '秦安县人民医院', 'hospital_property': '国营医院'}
{'hospital_rank': '一级甲等', 'hospital_name': '秦安县中医院', 'hospital_property': '国营医院'}
{'hospital_rank': 'None ', 'hospital_name': '秦安县中医医院', 'hospital_property': '国营医院'}
2561 303 甘谷县 天水市
http://www.a-hospital.com/w/天水市甘谷县医院列表
4
4
{'hospital_rank': '二级甲等', 'hospital_name': '甘谷县人民医院', 'hospital_property': '国营医院'}
{'hospital_rank': '二级乙等', 'hospital_name': '甘谷中医院', 'hospital_property': '国营医院'}
{'hospital_rank': '一级甲等', 'hospital_name': '甘谷澎仁医院', 'hospital_property': '股份制医院'}
{'hospital_rank': 'None ', 'hospital_name': '甘谷县中医医院', 'hospital_property': '国营医院'}
2562 303 武山县 天水市
http://www.a-hospital.com/w/天水市武山县医院列表
5
5
{'hospital_rank': '二级甲等', 'hospital_name': '武山县人民医院', 'hospital_property': '国营医院'}
{'hospital_rank': '一级甲等', 'hospital_name': '武山铜矿职工医院', 'hospital_property': '国营医院'}
{'hospital_rank': '一级甲等', 'hospital_name': '武山县中医院', 'hospital_prope

In [ ]:
##############################爬取 city_name+county_name 页面的医院数据

# -*- coding:utf-8 -*-
import requests
from bs4 import BeautifulSoup

url = 'http://www.a-hospital.com/w/北京市朝阳区医院列表'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
names = soup.select('ul > li > b > a')
#print(names)
shengshi = soup.select('ul > li > ul')
#print(shengshi)
print(len(names))
print(len(shengshi))


dict = {'hospital_name': 'None' ,'hospital_rank':'None ','hospital_property':'None'};
list=[]
#print(dict)
#print(list)



for i in range(len(names)):
    name = names[i].find_parent('b').get_text()
    #print(name)
    dict['hospital_name'] = name
    #print(dict['hospital_name'])
    info = shengshi[i].select('li')
    for li in info:
        if li.b:
            title = li.b.get_text()
            if title == "医院等级":
                value = li.contents[1].replace('：', '')
                #print(title,":",value)
                dict['hospital_rank'] = value
                #print(dict['hospital_rank'])
            elif title == "经营方式":
                value2 = li.contents[1].replace('：', '')
                #print(title,":",value2)
                dict['hospital_property'] = value2
                #print(dict['hospital_property'])
    #print(dict)
    list.append(dict);
    #dict={}
    dict = {'hospital_name': 'None' ,'hospital_rank':'None ','hospital_property':'None'};
    

    
#print(list)

In [ ]:
#存储海南省的市县代码 保存为：海南省.xls
#最新版本代码测试海南省
####################################海南省

str1='http://www.a-hospital.com/w/'
str3='医院列表'

L_city=[]
###（4,31）存储除了前四个直辖市后面的省
#存储 所有市的列表    其中该列表中的每一项包括：：一个市名，，该市下所有的区
for i in range(4,31):
    for j in range(1,len(L[i])):
        #print(L[i][j])
        L_city.append(L[i][j])  
    #print('\n')   
#print(L_city)
print(len(L_city))


str1='http://www.a-hospital.com/w/'
str3='医院列表'
county_info = []
#############################################################获取每个市下的  区

#13, 21, 17, 14, 11, 18, 21, 13, 11, 17, 14, 10, 11, 16, 9, 17, 14, 11, 9, 
#18, 12, 14, 9, 19, 8, 4, 7

###13个市，L_city中的顺序为（0到12） 起始数字：上一次的range()末尾数           起始数字+（个数） = range末尾数字


re_list=[]

for re in range(320,339):
    str2 = L_city[re]
    str=str1+str2+str3
    #print(str)
    html_city = requests.get(str).text
    soup_city = BeautifulSoup(html_city, 'lxml')
    county = soup_city.find('ul')
    #print(county.get_text())
    a_list = county.select('a')
    
    
    
    #print(len(a_list))
    #print(a_list[0].get_text())
    if(a_list[0].get_text() == '登录／创建账户'):
        re_list.append(re)
        #print(re_list)

id=0        
for new_re in range(320,339):
    if (new_re == re_list[id]):
        id=id+1
        new_str= str1 + '海南省'+ L_city[new_re] + str3
        html_city = requests.get(new_str).text
        soup_city = BeautifulSoup(html_city, 'lxml')
        county = soup_city.find('ul')
        #print(county.get_text())
        a_list = county.select('a')
    else:
        str2 = L_city[new_re]
        str=str1+str2+str3
        #print(str)
        html_city = requests.get(str).text
        soup_city = BeautifulSoup(html_city, 'lxml')
        county = soup_city.find('ul')
        #print(county.get_text())
        a_list = county.select('a')
    
    eachcity_county = []
    eachcity_county.append(L_city[new_re])
    for u in range(len(a_list)):
        eachcity_county.append(a_list[u].get_text()[:-2])

    #print(eachcity_county)
    county_info.append(eachcity_county)

#也去除页面不存在的情况
for info in range(len(county_info)):
    if (county_info[info][1] == '登录／创建'):
        del county_info[info][1]
        
    #去除甲乙丙丁错误  
    elif ((county_info[info][1] == '一甲') or (county_info[info][1] == '一乙') or (county_info[info][1] == '一丙') 
        or (county_info[info][1] == '二甲') or (county_info[info][1] == '二乙') or (county_info[info][1] == '二丙') 
        or (county_info[info][1] == '三甲') 
        or (county_info[info][1] == '三乙') 
        or (county_info[info][1] == '三丙')):
            del county_info[info][1:] 
        #for del_co in range(len(county_info[info])):
            #del county_info

print(county_info[0])
print(county_info[1])
print(county_info[2])
print(county_info[5])
print(county_info[16])
print(county_info[17])


import xlwt  
workbook=xlwt.Workbook(encoding='utf-8')  
booksheet=workbook.add_sheet('Sheet 1', cell_overwrite_ok=True)  

for i in range(len(county_info)):  
    for j in range(len(county_info[i])):  
        booksheet.write(i,j,county_info[i][j])  
workbook.save('海南省.xls') 